In [3]:
!pip install pandas torch
import os 
print(os.getcwd())
print("to make git track changes")

C:\Users\DELL\Desktop


In [22]:
!pip install transformers

In [23]:
import pandas as pd 
import re #regular expression uselful for text cleaning
import torch 
from transformers import AutoTokenizer,AutoModel

In [24]:
#if we are loading datasets from hugging face then 
!pip install datasets
#link to the test model im using https://huggingface.co/datasets/nyu-mll/glue/viewer/mrpc
from datasets import load_dataset
my_dataset=load_dataset("glue","mrpc")
#split the training and the test data 
train_data=my_dataset["train"]
test_data=my_dataset["test"]
#the dataset is sentence pairs so call the set of first senetences like sentence_1 , second set sentences_2 both of these are lists
sentence_1_array=train_data["sentence1"]
sentence_2_array=train_data["sentence2"]

    

In [25]:
#preprocessing data , basically the same thing s she had in the TDA from lingustic just not reading from disc also needed the extra downloads 
import nltk 
from nltk.corpus import stopwords 
from nltk.stem import PorterStemmer 
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [34]:
def pre_processed(sentences):
    cleaned=[]
    for i in sentences:
        tokens=nltk.word_tokenize(i)
        words=[stemmer.stem(word.lower()) for word in tokens  if word.isalpha() and word.lower() not in stop_words]
        cleaned.append(words)
    return cleaned
sentence_list = ["Hello! I'm"," learning NLP."]
print(pre_processed(sentence_list))
#print(pre_processed(sentence_1_array))
#print(pre_processed(sentence_2_array))

[['hello'], ['learn', 'nlp']]


In [38]:
#to compute the embeddings first we need to get the tokens for the cleaned strings 
from huggingface_hub import login
#use your own acess token 
from transformers import AutoTokenizer
tokenizer=AutoTokenizer.from_pretrained("google/gemma-7b")
#pre_processed returns a list of list of strings 
#tokenizer can only take string or a list of strings 
cleaned_text_1=pre_processed(sentence_array_1)
cleaned_text_2=pre_processed(sentence_array_2)
cleaned_text_into_encoder_1=["".join(tokens) for tokens in cleaned_text_1]
cleaned_text_into_encoder_1=["".join(tokens) for tokens in cleaned_text_2]

encodings=tokenizer(text=cleaned_text_into_encoder,padding=True,truncation=True)
#input ids are the integer representation of the tokens , the embeddings are the vectors that correspond to these integers in the lookup table 
#attention mask - reprsents paddings?
print(encodings.keys())


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


KeysView({'input_ids': [[0, 0, 2, 17534], [2, 1897, 8287, 20603]], 'attention_mask': [[0, 0, 1, 1], [1, 1, 1, 1]]})


In [ ]:
#computing embeddings 
input_ids=encodings["input_ids"]
attention_mask=encodings["attention_mask"]
from transformers import AutoModel
import torch 
model=Automodel.from_pre_trained("google/gemma-7b")
model.eval()
device="cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
with torch.no_grad(): 
    outputs = model(input_ids=input_ids.to(device),
                    attention_mask=attention_mask.to(device))
    token_embeddings = outputs.last_hidden_state

